In [ ]:
# Parameters
joblib_path = "path/to/data.joblib"  # <-- provide path
timestamp_col = "timestamp"          # column with timestamps
ema_fast_period = 12
ema_slow_period = 26


In [ ]:
import joblib
import numpy as np
import pandas as pd

from scr.check_data import prepare_time_series
import scr.indicators as ind
from scr.q_labels_matching import enrich_q_labels_trend_one_side
from scr.visualisation import plot_enriched_actions_one_side


In [ ]:
# Load and check data
raw_df = joblib.load(joblib_path)
df = prepare_time_series(raw_df, timestamp_col=timestamp_col)


In [ ]:
# Add indicators
close = df['Close'].to_numpy(np.float64)
high = df['High'].to_numpy(np.float64)
low = df['Low'].to_numpy(np.float64)
volume = df['Volume'].to_numpy(np.float64)

df['EMA_fast_raw'] = ind.ema_numba(close, ema_fast_period)
df['EMA_slow_raw'] = ind.ema_numba(close, ema_slow_period)
df['EMA_fast'] = ind.ema_numba_safe(close, ema_fast_period)
df['EMA_slow'] = ind.ema_numba_safe(close, ema_slow_period)
df['SMA_14'] = ind.sma_numba(close, 14)
df['ADX_14'] = ind.adx_numba(high, low, close, 14)
df['ATR_14'] = ind.atr_numba(high, low, close, 14)
df['RSI_14'] = ind.rsi_numba(close, 14)
macd, macd_signal, macd_hist = ind.macd_numba(close, 12, 26, 9)
df['MACD'] = macd
df['MACD_signal'] = macd_signal
df['MACD_hist'] = macd_hist
stoch_k, stoch_d = ind.stoch_numba(high, low, close, 14, 3)
df['STOCH_K'] = stoch_k
df['STOCH_D'] = stoch_d
boll_mid, boll_up, boll_dn = ind.bollinger_numba(close, 20, 2.0)
df['BOLL_MID'] = boll_mid
df['BOLL_UP'] = boll_up
df['BOLL_DN'] = boll_dn
df['OBV'] = ind.obv_numba(close, volume)
df['CCI_20'] = ind.cci_numba(high, low, close, 20)
df['WILLR_14'] = ind.williams_r_numba(high, low, close, 14)
df['MFI_14'] = ind.mfi_numba(high, low, close, volume, 14)
df['ROC_12'] = ind.roc_numba(close, 12)
df['VWAP_20'] = ind.vwap_numba(high, low, close, volume, 20)
piv = ind.zigzag_pivots_highlow_numba(high, low, close, 0.05, -0.05, 3, 1)
df['Pivots'] = piv
df['Pivots_expanded'] = ind.expand_pivots(piv, 2)


In [ ]:
# Signal marking via EMA cross
diff = df['EMA_fast'] - df['EMA_slow']
cross = np.sign(diff).diff().fillna(0)
df['Signal_Rule'] = np.select(
    [cross > 0, cross < 0],
    [1, -1],
    default=0,
)


In [ ]:
# Q-labels
df = enrich_q_labels_trend_one_side(df)


In [ ]:
# Visualisation
plot_enriched_actions_one_side(df)
